<a href="https://colab.research.google.com/github/S-15-77/notebook/blob/main/DataScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets accelerate evaluate


In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer

# Load the dataset
import pandas as pd

df_train = pd.read_csv("hf://datasets/osunlp/TravelPlanner/train.csv")
df_valid = pd.read_csv("hf://datasets/osunlp/TravelPlanner/validation.csv")
df_test = pd.read_csv("hf://datasets/osunlp/TravelPlanner/test.csv")
dataset = {
    "train": Dataset.from_pandas(df_train),
    "validation": Dataset.from_pandas(df_valid),
    "test": Dataset.from_pandas(df_test)
}

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 needs this manually set

# Format function (merge prompt + target)
def format_function(example):
    prompt = f"User Request: {example['query']}\nItinerary:"
    target = example['reference_information']
    return {"text": f"{prompt} {target}"}

# Apply formatting to each split
formatted_dataset = {
    split: dataset[split].map(format_function)
    for split in dataset
}

# Tokenization function
def tokenize_function(example):
    return tokenizer(example["text"], padding="max_length", truncation=True, max_length=512)

# Tokenize datasets
tokenized_dataset = {
    split: formatted_dataset[split].map(tokenize_function, batched=True)
    for split in formatted_dataset
}

# Set format for PyTorch
for split in tokenized_dataset:
    tokenized_dataset[split].set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
pip install -U transformers


In [ ]:
from transformers import AutoModelForCausalLM, Trainer, TrainingArguments

# Load model
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

# Training arguments
# For older transformers versions
training_args = TrainingArguments(
    output_dir="./gpt2-travelplanner",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=50
)



In [ ]:
def tokenize_function(example):
    tokens = tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    tokens["labels"] = tokens["input_ids"].copy()  # Important!
    return tokens


In [ ]:
# Re-tokenize all splits
tokenized_dataset = {
    split: formatted_dataset[split].map(tokenize_function, batched=True)
    for split in formatted_dataset
}

# Set torch format
for split in tokenized_dataset:
    tokenized_dataset[split].set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/45 [00:00<?, ? examples/s]

Map:   0%|          | 0/180 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
)

trainer.train()
trainer.save_model("gpt2-travelplanner")
tokenizer.save_pretrained("gpt2-travelplanner")


/tmp/ipython-input-27-3627668416.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

In [ ]:
huggingface-cli login


In [ ]:
model.push_to_hub("your-username/gpt2-travelplanner")
tokenizer.push_to_hub("your-username/gpt2-travelplanner")


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2-travelplanner", tokenizer="gpt2-travelplanner")

prompt = "User Request: 3-day trip to Tokyo with sushi and anime\nItinerary:"
output = generator(prompt, max_length=200, do_sample=True, temperature=0.9)
print(output[0]['generated_text'])


LM Model
/


In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer
from torch.optim import AdamW


# Use GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

#Extract as list of dicts (PyTorch-ready format)
train_data = [{
    "input_ids": torch.tensor(x["input_ids"]),
    "attention_mask": torch.tensor(x["attention_mask"]),
    "labels": torch.tensor(x["labels"]),
} for x in tokenized_dataset["train"]]

# Create DataLoader
train_loader = DataLoader(tokenized_dataset["train"], batch_size=4, shuffle=True)
#

ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

In [ ]:
optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
logging_interval = 50

model.train()
for epoch in range(num_epochs):
    total_loss = 0
    for step, batch in enumerate(train_loader):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if step % logging_interval == 0:
            print(f"Epoch {epoch+1}, Step {step}, Loss: {loss.item():.4f}")

    avg_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1} completed. Avg loss: {avg_loss:.4f}")


ValueError: Unable to avoid copy while creating an array as requested.
If using `np.array(obj, copy=False)` replace it with `np.asarray(obj)` to allow a copy when needed (no behavior change in NumPy 1.x).
For more details, see https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword.

In [ ]:
model.save_pretrained("gpt2-travelplanner")
tokenizer.save_pretrained("gpt2-travelplanner")


In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="gpt2-travelplanner", tokenizer="gpt2-travelplanner")
prompt = "User Request: 5-day trip to Tokyo focused on sushi and anime.\nItinerary:"
result = generator(prompt, max_length=200, do_sample=True, temperature=0.9)
print(result[0]["generated_text"])
